# How to handle WelDX files
In this notebook we will demonstrate how to create, read, and update ASDF files created by WelDX. All the needed funcationality is contained in a single class named `WeldxFile`. We are going to show different modes of operation, like working with physical files on your harddrive, and in-memory files, both read-only and read-write mode.

## Imports
The WeldxFile class is being imported from the top-level of the weldx package.

In [ ]:
from weldx import WeldxFile
import numpy as np
from datetime import datetime

## Basic operations
Now we create our first file, by invoking the `WeldxFile` constructor without any additional arguments. By doing so, we create an in-memory file. This means, that your changes will be temporary until you write it to an actual file on your harddrive. The `file_handle` attribute will point to the actual underlying file. In this case it is the in-memory file or buffer as shown below.

In [ ]:
file = WeldxFile()
file.file_handle

Next we assign some dictionary like data to the file, by storing it some attribute name enclosed by square brackets.
Then we look at the representation of the file header or contents. This will depend on the execution environment.
In JupyterLab you will see an interactive tree like structure, which can be expanded and searched.
The root of the tree is denoted as "root" followed by children created by the ASDF library "asdf_library" and "history". We attached the additional child "some_data" with our assignment.

In [ ]:
data = {"data_sets": {"first": np.random.random(100),
                      "time": datetime.now()}}

In [ ]:
file["some_data"] = data
file

Note, that here we are using some very common types, namely an NumPy array and a timestamp. For weldx specialized types like the coordinates system manager, (welding) measurements etc., the weldx package provides ASDF extensions to handle those types automatically during loading and saving ASDF data. You do not need to worry about them. If you try to save types, which cannot be handled by ASDF, you will trigger an error.

We could also have created the same structure in one step:

In [ ]:
file = WeldxFile(tree=data,
                 mode="rw")
file

You might have noticed, that we got a warning about the in-memory operation during showing the file in Jupyter.
Now we have passed the additional argument mode="rw", which indiciates, that we want to perform write operations just in memory,
or alternatively to the passed physical file. So this warning went away.

We can use all dictionary operations on the data we like, e.g. update, assign, and delete items.

In [ ]:
file["data_sets"]["second"] =  {"data": np.random.random(100),
                                "time": datetime.now()}

# delete the first data set again:
del file["data_sets"]["first"]
file

We can also iterate over all keys as usual. You can also have a look at the documentation of the builtin type `dict` for a complete overview of its features. 

In [ ]:
for key, value in file.items():
    print(key, value)

### Access to data by attributes
The access by key names can be tedious, when deeply nested dictionaries are involved. We provide a handling via attributes like this

In [ ]:
accessible_by_attribute = file.as_attr()
accessible_by_attribute.data_sets.second

## Writing files to disk
In order to make your changes persistent, we are going to save the memory-backed file to disk by invoking `WeldxFile.write_to`.

In [ ]:
file.write_to("example.asdf")

This newly created file can be opened up again, in read-write mode like by passing the appropriate arguments.

In [ ]:
example = WeldxFile("example.asdf", mode="rw")
example["updated"] = True
example.close()

Note, that we closed the file here explictly. Before closing, we wanted to write a simple item to tree. But lets see what happens, if we open the file once again.

In [ ]:
example = WeldxFile("example.asdf", mode='rw')
display(example)
example.close()

As you see the `updated` state has been written, because we closed the file properly. If we omit closing the file, 
our changes would be lost when the object runs out of scope or Python terminates.

## Handling updates within a context manager
To ensure you will not forget to update your file after making changes, 
we are able to enclose our file-changing operations within a context manager.
This ensures that all operations done in this context (the `with` block) are being written to the file, once the context is left.
Note that the underlying file is also closed after the context ends. This is useful, when you have to update lots of files, as there is a limited amount of file handles an operating system can deal with.

In [ ]:
with WeldxFile("example.asdf", mode='rw') as example:
    example["updated"] = True
    fh = example.file_handle
    # now the context ends, and the file is being saved to disk again.

# lets check the file handle has been closed, after the context ended.
assert fh.closed

Let us inspect the file once again, to see whether our `updated` item has been correctly written. 

In [ ]:
WeldxFile("example.asdf")

In case an error got triggered (e.g. an exception has been raised) inside the context, the underlying file is still updated. You could prevent this behavior, by passing `sync=False` during file construction.  

In [ ]:
try:
    with WeldxFile("example.asdf", mode="rw") as file:
        file["updated"] = False
        raise Exception("oh no")
except Exception as e:
    print("expected error:", e)

In [ ]:
WeldxFile("example.asdf")

## Keeping a log of changes when manipulating a file
It can become quite handy to know what has been done to file in the past. Weldx files provide a history log, in which arbitrary strings can be stored with time stamps and used software. We quickly run you through the process of adding history entries to your file.

In [ ]:
filename_hist = "example_history.asdf"
with WeldxFile(filename_hist, mode="rw") as file:
    file["some"] = "changes"
    file.add_history_entry("added some changes")

In [ ]:
WeldxFile(filename_hist)["history"]

When you want to describe a custom software,
which is lets say a library or tool used to generate/modify the data in the file and we passed it into the creation of our WeldxFile.

In [ ]:
software = dict(name="my_tool", version="1.0", homepage="https://my_tool.org", author="the crowd")
with WeldxFile(filename_hist, mode="rw", software_history_entry=software) as file:
    file["some"] = "changes"
    file.add_history_entry("added more changes")

Lets now inspect how we wrote history.

In [ ]:
WeldxFile(filename_hist)["history"]["entries"][-1]

The entries key is a list of all log entries, where new entries are appended to. We have proper time stamps indicating when the change happened, the actual log entry, and optionally a custom software used to make the change.

## Handeling of custom schemas
An important aspect of WelDX or ASDF files is, that you can validate them to comply with a defined schema. A schema defines required and optional attributes a tree structure has to provide to pass the schema validation. Further the types of these attributes can be defined, e.g. the data attribute should be a NumPy array, or a timestamp should be of type `pandas.Timestamp`.
There are several schemas provided by WelDX, which can be used by passing them to the `custom_schema` argument. It is expected to be a path-like type, so a string (`str`) or `pathlib.Path` is accepted. The provided utility function `get_schema_path` returns the path to named schema. So its output can directly be used in WeldxFile(schema=...)

In [ ]:
from weldx.asdf.util import get_schema_path

In [ ]:
schema = get_schema_path("single_pass_weld-1.0.0")
schema

This schema defines a complete experimental setup with measurement data, e.g requires the following attributes to be defined in our tree:

  - workpiece
  - TCP
  - welding_current
  - welding_voltage
  - measurements
  - equipment

We use a testing function to provide this data now, and validate it against the schema by passing the `custom_schema` during WeldxFile creation.
Here we just have a look at the process parameters sub-dictionary.

In [ ]:
from weldx.asdf.cli.welding_schema import single_pass_weld_example
_, single_pass_weld_data = single_pass_weld_example(out_file=None)
display(single_pass_weld_data["process"])

That is a lot of data, containing complex data structures and objects describing the whole experiment including measurement data.
We can now create new `WeldxFile` and validate the data against the schema.

In [ ]:
WeldxFile(tree=single_pass_weld_data, custom_schema=schema, mode='rw')

But what would happen, if we forget an import attribute? Lets have a closer look...

In [ ]:
# simulate we forgot something important, so we delete the workpiece:
del single_pass_weld_data["workpiece"]

# now create the file again, and see what happens:
try:
    WeldxFile(tree=single_pass_weld_data, custom_schema=schema, mode='rw')
except Exception as e:
    display(e)

We receive a ValidationError from the ASDF library, which tells us exactly what the missing information is. The same will happen, if we accidentially pass the wrong type.

In [ ]:
# simulate a wrong type by changing it to a NumPy array.
single_pass_weld_data["welding_current"] = np.zeros(10)

# now create the file again, and see what happens:
try:
    WeldxFile(tree=single_pass_weld_data, custom_schema=schema, mode='rw')
except Exception as e:
    display(e)

Here we see, that a `signal` tag is expected, but a `asdf/core/ndarray-1.0.0` was received. 
The ASDF library assignes tags to certain types to handle their storage in the file format. 
As shown, the `signal` tag is contained in `weldx/measurement` container, provided by `weldx.bam.de`. The tags and schemas also provide a version number, so future updates in the software become managable.

Custom schemas can be used to define own protocols or standards describing your data.

## Summary
In this tutorial we have encountered how to easily open, inspect, manipulate, and update ASDF files created by WelDX. We've learned that these files can store a variety of different data types and structures.

Discussed features:

 * Opening in read/write mode `WeldxFile(mode="rw")`.
 * Creating files in memory (passing no file name to `WeldxFile()` constructor).
 * Writing to disk (`WeldxFile.write_to`).
 * Keeping log of changes (`WeldxFile.history`, `WeldxFile.add_history_entry`).
 * Validation against a schema `WeldxFile(custom_schema="/path/my_schema.yaml")`